## Split-Apply-Combine

It's pretty common to want to ask questions like

-  What is the average culmen length of the penguins living on the Island Torgersen?
-  Does the average culmen depth differ from one species to another?


One of the fundamental tasks in exploratory data analysis is to summarize your data **by group**. In our penguins data, for example, a very natural thing to do is to compute summary statistics **by species**, or perhaps by habitat (or both!). We can contextualize this task in three stages: 

1. **Split** the data data frame into pieces, one for each species. 
2. **Apply** an aggregation function to each piece, yielding a single number. 
3. **Combine** the results into a new data frame.

This pattern is so common that the phrase "split-apply-combine" now appears in many texts on data analysis. This phrase was originally coined by Hadley Wickham, who is famous for developing many of the modern tools for data analysis in the `R` programming language. 

<figure class="image" style="width:50%">
  <img src="https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png" alt="Left: A single dataframe is split into three pieces. Middle: The data within each piece is summed. Right: the resulting sums are combined, resulting in a new data frame with one sum for each piece.">
  <figcaption><i>split-apply-combine. Image credit: Jake VanderPlas, in the Python Data Science Handbook</i></figcaption>
</figure>

### Python lets us easily perform split-apply-combine operations using the `groupby()` method of data frames. 

In [18]:
# Same preprocessing as last video 

import pandas as pd
import numpy as np

#read in data from csv
penguins=pd.read_csv("palmer_penguins.csv")

cols=["Species", "Region", "Island", "Culmen Length (mm)", "Culmen Depth (mm)"]
penguins=penguins[cols]

penguins["Species"]=penguins["Species"].str.split().str.get(0)

penguins["Length"]=penguins["Culmen Length (mm)"]
penguins["Depth"]=penguins["Culmen Depth (mm)"]

penguins = penguins.drop(labels=["Culmen Depth (mm)","Culmen Length (mm)"],axis=1)
penguins.head()

,Species,Region,Island,Length,Depth
0,Adelie,Anvers,Torgersen,39.1,18.7
1,Adelie,Anvers,Torgersen,39.5,17.4
2,Adelie,Anvers,Torgersen,40.3,18.0
3,Adelie,Anvers,Torgersen,NaN,NaN
4,Adelie,Anvers,Torgersen,36.7,19.3


We can group by species with the groupby() method

In [11]:
penguins.groupby("Species")

Now, we can get the mean of each species by adding .mean()

In [12]:
penguins.groupby("Species").mean()

,Length,Depth
Species,,
Adelie,38.791391,18.346358
Chinstrap,48.833824,18.420588
Gentoo,47.504878,14.982114


If we only want the mean of the Length column, we add in [["Length"]](Note the double brackets)

In [13]:
penguins.groupby("Species")[["Length"]].mean()

,Length
Species,
Adelie,38.791391
Chinstrap,48.833824
Gentoo,47.504878


We can get multiple summary statistics e.g., mean and standard deviation together via the aggreate() method

In [14]:
penguins.groupby("Species").aggregate([np.mean,np.std])
#note there was no () in the function names

Length                Depth          
                mean       std       mean       std
Species                                            
Adelie     38.791391  2.663405  18.346358  1.216650
Chinstrap  48.833824  3.339256  18.420588  1.135395
Gentoo     47.504878  3.081857  14.982114  0.981220

### Group by multiple columns at the same time 

Group by Species and Island

In [15]:
summary=penguins.groupby(["Species","Island"]).aggregate([np.mean,np.std])
summary

Length                Depth          
                          mean       std       mean       std
Species   Island                                             
Adelie    Biscoe     38.975000  2.480916  18.370455  1.188820
          Dream      38.501786  2.465359  18.251786  1.133617
          Torgersen  38.950980  3.025318  18.429412  1.339447
Chinstrap Dream      48.833824  3.339256  18.420588  1.135395
Gentoo    Biscoe     47.504878  3.081857  14.982114  0.981220

Hierarchical Indexing
Complex data summary tables like the one above are useful and powerful, but they also pose an important problem -- how can we extract the data from these summaries? For example, how can I get the mean bill length for Chinstrap penguins on Dream island? To extract this kind of data, we need to use hierarchical indexing, in which we pass multiple keys to the .loc attribute. After passing all the row indices, we need to use .loc again to get at the column indices.

In [16]:
#first restrict to adelie, dream
adelie_dream=summary.loc["Adelie","Dream"]
adelie_dream

Length  mean    38.501786
        std      2.465359
Depth   mean    18.251786
        std      1.133617
Name: (Adelie, Dream), dtype: float64

In [17]:
#now grab the mean of the length column
adelie_dream.loc["Length", "mean"]

38.501785714285695